<a href="https://colab.research.google.com/github/charlieswb/Learning/blob/master/Dota2_predict_winning_team.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df = pd.read_csv('/content/dota_games.txt', header = None)
df = df.rename(columns={0: 'radiant_1', 1: 'radiant_2', 2: 'radiant_3', 3: 'radiant_4', 4: 'radiant_5',\
                        5: 'dire_1', 6: 'dire_2', 7: 'dire_3', 8: 'dire_4', 9: 'dire_5', 10: 'team_win'})

In [ ]:
df.loc[df['team_win']==1, 'result'] = 'Radiant'
df.loc[df['team_win']==2, 'result'] = 'Dire'
df = df.drop(['team_win'], axis=1)
df.head(10)

,radiant_1,radiant_2,radiant_3,radiant_4,radiant_5,dire_1,dire_2,dire_3,dire_4,dire_5,result
0,Sven,Lone Druid,Venomancer,Clockwerk,Shadow Shaman,Invoker,Gyrocopter,Anti-Mage,Alchemist,Slark,Dire
1,Riki,Tinker,Puck,Leshrac,Nyx Assassin,Slardar,Sand King,Spectre,Necrolyte,Warlock,Radiant
2,Invoker,Mirana,Pudge,Magnus,Keeper of the Light,Rubick,Tidehunter,Queen of Pain,Faceless Void,Sniper,Dire
3,Riki,Centaur Warrunner,Treant Protector,Queen of Pain,Broodmother,Rubick,Weaver,Troll Warlord,Alchemist,Drow Ranger,Radiant
4,Razor,Kunkka,Drow Ranger,Leshrac,Zeus,Riki,Bane,Visage,Invoker,Timbersaw,Radiant
5,Sand King,Shadow Shaman,Silencer,Gyrocopter,Meepo,Nyx Assassin,Slark,Mirana,Rubick,Queen of Pain,Dire
6,Morphling,Omniknight,Pudge,Crystal Maiden,Razor,Troll Warlord,Tidehunter,Lina,Dark Seer,Gyrocopter,Radiant
7,Brewmaster,Leshrac,Ogre Magi,Kunkka,Riki,Disruptor,Juggernaut,Tiny,Enigma,Bounty Hunter,Radiant
8,Slardar,Earthshaker,Invoker,Slark,Doom,Necrolyte,Timbersaw,Bounty Hunter,Huskar,Razor,Radiant
9,Dazzle,Naga Siren,Ogre Magi,Pudge,Nature's Prophet,Chen,Puck,Timbersaw,Magnus,Ursa,Dire


In [ ]:
X = df.drop(['result'], axis=1)
labels = df['result']
Y = pd.get_dummies(labels, dtype='float32').values

In [ ]:
r = X.iloc[:,:5]
d = X.iloc[:,5:]
r = r.stack().str.get_dummies().sum(level=0).astype('float32')
d = d.stack().str.get_dummies().sum(level=0).astype('float32')

In [ ]:
X = r.values - d.values
print(X.shape)
X

(15000, 97)


array([[-1.,  0., -1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32)

In [ ]:
class Classifier(keras.Model):
  def __init__(self):
    super().__init__()
    self.l1 = Dense(97)
    self.l2 = Dense(97)
    self.l3 = Dense(97)
    self.l4 = Dense(97)
    self.l5 = Dense(97)
    self.l6 = Dense(97)
    self.l7 = Dense(97)
    self.l8 = Dense(97)
    self.l9 = Dense(97)
    self.l10 = Dense(2)
  def call(self, inputs):
    x = self.l1(inputs)
    x = self.l2(x)
    x = self.l3(x)
    x = self.l4(x)
    x = self.l5(x)
    x = self.l6(x)
    x = self.l7(x)
    x = self.l8(x)
    x = self.l9(x)
    x = self.l10(x)
    return x

In [ ]:
best_model_loss = 1
n_epochs = 300
kfold = StratifiedKFold(n_splits=6, shuffle=True)

for train_index,test_index in kfold.split(X, labels):
  X_train, X_test = X[train_index], X[test_index]
  Y_train, Y_test = Y[train_index], Y[test_index]
  
  model = Classifier()
  model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.0001), loss=keras.losses.MeanSquaredError()) 
  model.fit(X_train, Y_train, epochs=200, verbose=0)
  
  loss = model.evaluate(X_test, Y_test)
  if loss < best_model_loss:
    best_model = model
    best_model_loss = loss
  print('loss:', loss)

79/79 [==============================] - 0s 2ms/step - loss: 0.2500
loss: 0.2499718815088272
79/79 [==============================] - 0s 1ms/step - loss: 0.2499
loss: 0.24985751509666443
79/79 [==============================] - 0s 1ms/step - loss: 0.2515
loss: 0.251483678817749
79/79 [==============================] - 0s 1ms/step - loss: 0.2500
loss: 0.25000637769699097
79/79 [==============================] - 0s 1ms/step - loss: 0.2500
loss: 0.25001755356788635
79/79 [==============================] - 0s 1ms/step - loss: 0.2497
loss: 0.2496758997440338


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=33)

In [ ]:
model = Classifier()
model.compile(optimizer=keras.optimizers.SGD(learning_rate=0.00009), loss=keras.losses.MeanSquaredError())

In [ ]:
model.fit(X_train, Y_train, epochs=20, verbose=0)

In [ ]:
print('loss:', model.evaluate(X_test, Y_test))

94/94 [==============================] - 0s 1ms/step - loss: 0.2522
loss: 0.2521933913230896
